In [1]:
import polars_ds as pld
import polars as pl
import numpy as np 

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.375726,0.938769,0,10000,-10000,0,0.025604,"""a"""
0.841471,1,"""a""",0.330279,0.089778,1,10001,-9999,1,0.375945,"""a"""
0.909297,2,"""a""",0.505979,0.986331,2,10002,-9998,1,0.116006,"""a"""
0.14112,3,"""a""",0.535027,0.984207,3,10003,-9997,1,0.339372,"""a"""
-0.756802,4,"""a""",0.045169,0.117995,4,10004,-9996,0,0.850384,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pl.col("x1").num.jaccard(pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
df.select(
    pl.col("f").num.rfft()
).head()

f
list[f64]
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# Least Square (Linear Regression)
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)

y
list[f64]
"[2.0, -1.0]"


In [6]:
df.lazy().select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
).collect()

y
list[f64]
"[2.0, -1.0]"


In [7]:
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).over(pl.col("dummy"))
).head() 

coeffs
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [8]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False, return_pred=True).alias("pred")
).unnest("pred").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,0.0
1,10001,-9999,-9999.0,0.0
2,10002,-9998,-9998.0,0.0
3,10003,-9997,-9997.0,0.0
4,10004,-9996,-9996.0,0.0


In [9]:
df.group_by("dummy").agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)


dummy,coeffs
str,list[f64]
"""b""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [10]:
# Rolling regression
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [11]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pl.col("y").num.cond_entropy(pl.col("x1"))
)

y
f64
-0.0


In [12]:
df.group_by("dummy_groups").agg(
    pl.col("actual").metric.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").metric.bce(pl.col("predicted")).alias("log loss"),
    pl.col("actual").metric.binary_metrics_combo(pl.col("predicted")).alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.331987,0.988272,0.501209,0.497004,0.249549,0.505541,0.503486
"""a""",0.335586,0.998342,0.511257,0.486181,0.249202,0.514072,0.498852


# Str Extension

In [13]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [14]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""to"""
"""church"""
"""world"""
"""going"""
"""hello"""


In [15]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
"""world"""
"""hello"""
"""church"""
"""go"""


In [16]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [17]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [18]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [19]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pl.col("word").str2.levenshtein_filter("world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [20]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [21]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [22]:

df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,null,0.833333,0.966667
0.428571,0.333333,0.272727,null,0.0,0.444444
0.111111,0.111111,0.090909,null,0.444444,0.5
0.875,0.666667,0.545455,null,0.25,0.527778
0.75,0.777778,0.454545,null,0.25,0.527778


# Stats Extension

In [23]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.365573
-1.010568
0.954965


In [24]:
# Genenrate random sample, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
)

a,random
f64,f64
null,null
null,null
0.365573,0.075261
-1.010568,-1.628971
0.954965,0.662764
-0.052829,0.316189
-0.324009,0.596851
0.704042,0.965986
-0.113048,1.833017


In [25]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
)

a,random_str
f64,str
null,null
null,null
0.365573,"""Z"""
-1.010568,"""zflx"""
0.954965,"""b"""
-0.052829,"""Z"""
-0.324009,"""E"""
0.704042,"""mg"""
-0.113048,"""D"""


In [26]:
# Genenrate fixed size random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
)

a,random_str
f64,str
null,null
null,null
0.365573,"""msFtE"""
-1.010568,"""3JYfz"""
0.954965,"""bHt5D"""
-0.052829,"""5Qg37"""
-0.324009,"""gKmxk"""
0.704042,"""Wytqn"""
-0.113048,"""8x6EK"""


In [27]:
df.with_columns(
    # Sample from normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.sample_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.sample_uniform(low = 0., high = None, respect_null=True).alias("test2")
).head()

a,test1,test2
f64,f64,f64
null,-0.252794,null
null,0.765073,null
0.365573,-0.558877,2.302708
-1.010568,-1.827438,2.435242
0.954965,0.659293,1.666999


In [28]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1.).alias("test1")
    , pl.col("a").stats.sample_normal(mean = 0.5, std = 2.).alias("test2")
).select(
    pl.col("test1").stats.ttest_ind(pl.col("test2"), equal_var = False).alias("t-test")
    , pl.col("test1").stats.normal_test().alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
0.91856,0.358475,3.487495,0.174864


In [29]:
size = 5000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.728025,0.077981,4,3
1,0.135723,0.274663,0,2
2,0.983201,0.829362,3,4
0,0.931766,0.292064,4,7
1,0.269802,0.912201,4,3


In [30]:
# In dataframe statistical tests!
print(df.select(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
))

shape: (1, 3)
┌──────────────────────┬──────────────────────┬─────────────────────┐
│ t-test               ┆ chi2-test            ┆ f-test              │
│ ---                  ┆ ---                  ┆ ---                 │
│ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞══════════════════════╪══════════════════════╪═════════════════════╡
│ {-0.661944,0.508023} ┆ {31.809507,0.668211} ┆ {1.289306,0.271702} │
└──────────────────────┴──────────────────────┴─────────────────────┘


In [31]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = False).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),-
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{-0.578175,0.563185}","{19.636509,0.987899}","{-1.16222,-0.325678}"
1,"{-0.145807,0.884083}","{30.663435,0.720159}","{-0.9458,-0.43643}"
2,"{-0.420535,0.674122}","{48.858384,0.074686}","{-1.014709,-0.398437}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [32]:
import polars_ds as pld
df = pl.DataFrame({
    "id": range(1000),
    "val1": np.random.random(size=1000), 
    "val2": np.random.random(size=1000), 
    "val3": np.random.random(size=1000),
    "r": np.random.random(size=1000),
    "rh": np.random.random(size=1000)*10,
})

In [33]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.select(
    pld.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", # L Infinity distance 
        parallel = True 
    )
).head()

""
u32
5
3
8
5
14


In [34]:
df.select(
    pld.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "l1", # L 1 distance 
        parallel = True 
    )
).head()

r
u32
92
480
1
397
142


In [35]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pl.col("id").num.knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends")
).head()

id,val1,val2,val3,r,rh,best friends
i64,f64,f64,f64,f64,f64,list[u64]
0,0.518424,0.678108,0.189745,0.934401,6.533454,"[0, 177, … 848]"
1,0.355768,0.922746,0.044014,0.759396,8.07259,"[1, 807, … 221]"
2,0.516528,0.648056,0.813823,0.086277,4.768204,"[2, 827, … 772]"
3,0.313269,0.832757,0.261585,0.94688,6.00855,"[3, 564, … 87]"
4,0.232905,0.465452,0.165761,0.96089,8.762937,"[4, 174, … 482]"


In [36]:
# Filter to only points near the given point
df.filter(
    pld.query_radius(
        [0.5, 0.5, 0.5],
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        radius = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
0,0.518424,0.678108,0.189745,0.934401,6.533454
2,0.516528,0.648056,0.813823,0.086277,4.768204
4,0.232905,0.465452,0.165761,0.96089,8.762937
7,0.564561,0.814784,0.43214,0.330617,6.106174
9,0.727656,0.428819,0.65269,0.787735,7.677925


In [37]:
# Haversine distance is available when dimension is 2
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        radius = 10, # in km
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
45,0.525377,0.470926,0.940175,0.005257,4.37334
109,0.5147,0.585091,0.050174,0.011534,9.686307
111,0.482627,0.444948,0.3163,0.691749,6.042239
120,0.496,0.574053,0.9983,0.679868,6.184419
130,0.466521,0.542723,0.274116,0.669029,1.118131


In [38]:
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        # Columns used as the coordinates in n-d space
        pl.col("val1"), pl.col("val2"), 
        # radius can also be an existing column in the dataframe.
        radius = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
45,0.525377,0.470926,0.940175,0.005257,4.37334
109,0.5147,0.585091,0.050174,0.011534,9.686307
190,0.455379,0.517141,0.674352,0.384756,9.140035
239,0.575073,0.518422,0.07625,0.764358,9.744551
246,0.490666,0.516131,0.185814,0.259276,6.761479


# String Nearest Neighbors

This might be slow for very large vocab / column.

In [39]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"]
})

In [40]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 1,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [41]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [42]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [43]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
